Found from here:
https://www.kaggle.com/code/fehiepsi/bayesian-imputation-for-age/notebook

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In this notebook, we will do logistic regression to predict `Survived` using `Age` variable. For simplicity, I'll skip EAD part (which has been nicely done in many other popular kernels). I'll use [NumPyro](https://github.com/pyro-ppl/numpyro) for modelling, sampling, and making predictions.

In [2]:
#!pip install numpyro

In [3]:
from jax import ops, random
from jax.scipy.special import expit

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

### prepare data

After loading data, I recognize that there are many missing values for `Age` column. We know (by intuition or from other kernels) that `Age` is correlated with the title of the name: e.g. those with `Mrs.` would be older than those with `Miss.`. Let's make a new column `Title` for that purpose.

In [4]:
train_df = pd.read_csv(
    "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv"
)
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#train_df = pd.read_csv("../input/titanic/train.csv")
d = train_df.copy()
d.Embarked.fillna("S", inplace=True)  # filling 2 missing data points with the mode "S"
d["Title"] = d.Name.str.split(", ").str.get(1).str.split(" ").str.get(0).apply(
    lambda x: x if x in ["Mr.", "Miss.", "Mrs.", "Master."] else "Misc.")
title_cat = pd.CategoricalDtype(categories=["Mr.", "Miss.", "Mrs.", "Master.", "Misc."], ordered=True)
age_mean, age_std = d.Age.mean(), d.Age.std()
embarked_cat = pd.CategoricalDtype(categories=["S", "C", "Q"], ordered=True)
data = dict(age=d.Age.pipe(lambda x: (x - age_mean) / age_std).values,
            pclass=d.Pclass.values - 1,
            title=d.Title.astype(title_cat).cat.codes.values,
            sex=(d.Sex == "male").astype(int).values,
            sibsp=d.SibSp.clip(0, 1).values,
            parch=d.Parch.clip(0, 2).values,
            embarked=d.Embarked.astype(embarked_cat).cat.codes.values,
            survived=d.Survived.values)

Note that I don't use other features such as `Fare` or `Cabin` for simplicity. I also don't do much of feature engineering for the same reason.

### modelling

If you are not familiar with NumPyro, you can take a look at [its documentation](https://github.com/pyro-ppl/numpyro#numpyro) which includes some tutorials, examples, and translated code for Statistical Rethinking book (which is a good reference IMO if you are not familiar with Bayesian methods).

In [6]:
def model(age, pclass, title, sex, sibsp, parch, embarked, survived=None):
    # create a variable for each of Pclass, Title, Sex, SibSp, Parch,
    b_pclass = numpyro.sample("b_Pclass", dist.Normal(0, 1), sample_shape=(3,))
    b_title = numpyro.sample("b_Title", dist.Normal(0, 1), sample_shape=(5,))
    b_sex = numpyro.sample("b_Sex", dist.Normal(0, 1), sample_shape=(2,))
    b_sibsp = numpyro.sample("b_SibSp", dist.Normal(0, 1), sample_shape=(2,))
    b_parch = numpyro.sample("b_Parch", dist.Normal(0, 1), sample_shape=(3,))
    b_embarked = numpyro.sample("b_Embarked", dist.Normal(0, 1), sample_shape=(3,))

    # impute Age by Title
    age_mu = numpyro.sample("age_mu", dist.Normal(0, 1), sample_shape=(5,))
    age_mu = age_mu[title]
    age_sigma = numpyro.sample("age_sigma", dist.Normal(0, 1), sample_shape=(5,))
    age_sigma = age_sigma[title]
    age_isnan = np.isnan(age)
    age_nanidx = np.nonzero(age_isnan)[0]
    if survived is not None:
        age_impute = numpyro.param("age_impute", np.zeros(age_isnan.sum()))
    else:  # for prediction, we sample `age_impute` from Normal(age_mu, age_sigma)
        age_impute = numpyro.sample("age_impute", dist.Normal(age_mu[age_nanidx], age_sigma[age_nanidx]))
    age = ops.index_update(age, age_nanidx, age_impute)
    numpyro.sample("age", dist.Normal(age_mu, age_sigma), obs=age)

    a = numpyro.sample("a", dist.Normal(0, 1))
    b_age = numpyro.sample("b_Age", dist.Normal(0, 1))
    logits = a + b_age * age

    logits = logits + b_title[title] + b_pclass[pclass] + b_sex[sex] \
        + b_sibsp[sibsp] + b_parch[parch] + b_embarked[embarked]
    # for prediction, we will convert `logits` to `probs` and record that result
    if survived is None:
        probs = expit(logits)
        numpyro.sample("probs", dist.Delta(probs))
    numpyro.sample("survived", dist.Bernoulli(logits=logits), obs=survived)

### sampling

After making a model, sampling is pretty fast in NumPyro.

In [7]:
mcmc = MCMC(NUTS(model), num_warmup=1000, num_samples=1000)
#mcmc = MCMC(NUTS(model), 1000, 1000)
mcmc.run(random.PRNGKey(0), **data)
mcmc.print_summary()

sample: 100%|█| 2000/2000 [00:19<00:00, 103.61it/s, 63 steps of size 6.77e-02. a



                   mean       std    median      5.0%     95.0%     n_eff     r_hat
            a      0.10      0.82      0.10     -1.17      1.53    896.51      1.00
    age_mu[0]      0.14      0.03      0.14      0.08      0.20   1942.02      1.00
    age_mu[1]     -0.44      0.06     -0.43     -0.55     -0.35   2463.83      1.00
    age_mu[2]      0.37      0.07      0.37      0.25      0.49   2231.83      1.00
    age_mu[3]     -1.54      0.09     -1.54     -1.69     -1.40   1906.34      1.00
    age_mu[4]      0.82      0.18      0.81      0.52      1.09   1409.18      1.00
 age_sigma[0]      0.77      0.02      0.77      0.73      0.81   1669.72      1.00
 age_sigma[1]      0.83      0.04      0.83      0.75      0.89   1569.06      1.00
 age_sigma[2]      0.75      0.05      0.75      0.68      0.83   1495.08      1.00
 age_sigma[3]      0.59      0.07      0.59      0.47      0.69   1562.45      1.00
 age_sigma[4]      0.94      0.14      0.92      0.72      1.15   1555.64  

As we can see, using Bayesian, we can get uncertainties of our results: e.g. imputing values, coefficients of being male or female,... (and you can make nice plots with them ;)

### make predictions

To make predictions on the new data, we will maginalize those `age_imput` variables (in other words, removing them from posterior samples) and use the remaining variables for predictions.

In [8]:
test_df = pd.read_csv(
    "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv"
)
test_df.info()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
#test_df = pd.read_csv("../input/titanic/test.csv")
d = test_df.copy()
d["Title"] = d.Name.str.split(", ").str.get(1).str.split(" ").str.get(0).apply(
    lambda x: x if x in ["Mr.", "Miss.", "Mrs.", "Master."] else "Misc.")
test_data = dict(age=d.Age.pipe(lambda x: (x - age_mean) / age_std).values,
                 pclass=d.Pclass.values - 1,
                 title=d.Title.astype(title_cat).cat.codes.values,
                 sex=(d.Sex == "male").astype(int).values,
                 sibsp=d.SibSp.clip(0, 1).values,
                 parch=d.Parch.clip(0, 2).values,
                 embarked=d.Embarked.astype(embarked_cat).cat.codes.values)

posterior = mcmc.get_samples().copy()
print(posterior)
posterior.pop("age_impute")
survived_probs = Predictive(model, posterior).get_samples(random.PRNGKey(2), **test_data)["probs"]
d["Survived"] = (survived_probs.mean(axis=0) >= 0.5).astype(np.uint8)
d[["PassengerId", "Survived"]].to_csv("submission.csv", index=False)

{'a': DeviceArray([ 3.75811636e-01, -7.07964674e-02, -5.19075513e-01,
              7.60513783e-01,  3.53536189e-01,  2.43986294e-01,
              1.25935555e+00, -1.00988495e+00, -1.61407933e-01,
              2.68858254e-01, -1.06434375e-01, -1.49172351e-01,
             -2.43900344e-01,  1.18554914e+00, -7.32230127e-01,
              7.46217012e-01,  4.88323420e-01, -7.09419012e-01,
              1.36485064e+00,  6.72040164e-01,  6.25139654e-01,
              5.07822931e-01, -3.89957100e-01, -7.60192394e-01,
             -8.16246271e-02,  2.39156947e-01, -1.00394748e-01,
             -1.47606373e+00,  1.17613578e+00,  2.75839043e+00,
              2.68628621e+00, -9.65978384e-01,  7.35919356e-01,
             -5.50252736e-01, -1.01845825e+00, -6.55398369e-01,
              8.32260028e-02,  1.07590127e+00,  3.21022898e-01,
             -8.67530778e-02, -1.30007938e-01, -1.42553771e+00,
             -9.50518072e-01,  1.00953376e+00,  1.43932745e-01,
              6.64997399e-01,  3.1

KeyError: 'age_impute'

Submiting the result gives me the score about 79 (top 16%). It is great for the first attempt. :)

### further improvements

+ Using other features such as `Cabin` or `Fare`.
+ The above model assumes a linear relationship of `Survived` w.r.t. other latent variables. The result is intuitive but is not enough to beat tree-based models. We can build more complicated models or construct a [Bayesian neural network](http://pyro.ai/numpyro/bnn.html) model to capture more complex relationship.